In [1]:
#!pip install pmlb
import pandas as pd
import numpy as np
from pmlb import fetch_data

In [2]:
# dataset for this demo
#https://github.com/EpistasisLab/penn-ml-benchmarks/tree/master/datasets/classification/GAMETES_Epistasis_2-Way_20atts_0.4H_EDM-1_1
gametes = fetch_data('GAMETES_Epistasis_2-Way_20atts_0.4H_EDM-1_1')
print(gametes.describe())

                N0           N1           N2           N3           N4  \
count  1600.000000  1600.000000  1600.000000  1600.000000  1600.000000   
mean      0.020625     0.261250     0.728750     0.889375     0.354375   
std       0.142170     0.475015     0.673949     0.702635     0.523416   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       0.000000     0.000000     0.000000     0.000000     0.000000   
50%       0.000000     0.000000     1.000000     1.000000     0.000000   
75%       0.000000     0.000000     1.000000     1.000000     1.000000   
max       1.000000     2.000000     2.000000     2.000000     2.000000   

                N5           N6           N7           N8           N9  ...  \
count  1600.000000  1600.000000  1600.000000  1600.000000  1600.000000  ...   
mean      0.791250     0.502500     0.728750     0.346875     0.761875  ...   
std       0.695864     0.615614     0.684994     0.540125     0.694062  ...   
min       0.00000

In [3]:
gametes.head()

,N0,N1,N2,N3,N4,N5,N6,N7,N8,N9,...,N11,N12,N13,N14,N15,N16,N17,P1,P2,target
0,0,0,2,1,1,0,0,2,0,1,...,2,1,0,0,0,0,0,0,1,1
1,0,0,1,0,0,1,1,0,0,1,...,0,1,0,0,0,0,0,0,0,1
2,0,0,0,1,0,2,0,0,0,0,...,1,1,0,0,2,2,0,0,0,1
3,0,1,0,2,0,1,0,2,0,0,...,1,1,0,1,1,0,1,1,0,1
4,0,0,1,1,0,0,0,1,1,0,...,1,0,0,1,0,0,1,1,0,1


In [4]:
X=gametes.drop('target',axis=1)

# features for A
A_feat_list = [
                'N10',
                'N11',
                'N12'
                ]

# features for C
C_feat_list = [
                'N13',
                'N14',
                'N15'
                ]
# y

y = gametes['target']

In [5]:
# test metaestimator alone
from tpot.builtins import MetaEstimator
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score, train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
metaest = MetaEstimator(estimator=GradientBoostingRegressor(random_state=42), A=';'.join(A_feat_list), C=';'.join(C_feat_list))
metaest.fit(X_train, y_train)
metaest.values_list

['ternary',
 'ternary',
 'ternary',
 'ternary',
 'ternary',
 'ternary',
 'ternary',
 'ternary',
 'ternary',
 'ternary',
 'ternary',
 'ternary',
 'ternary',
 'ternary']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train.head()

,N0,N1,N2,N3,N4,N5,N6,N7,N8,N9,N10,N11,N12,N13,N14,N15,N16,N17,P1,P2
820,0,0,0,2,1,1,0,1,1,0,0,1,1,1,0,0,1,1,1,1
567,0,0,0,0,0,0,0,0,0,0,0,2,2,0,1,0,0,2,0,1
355,0,1,0,1,1,1,1,1,0,1,0,1,2,0,0,0,1,0,0,0
1545,0,0,0,2,1,0,0,1,1,2,0,0,0,0,0,1,2,1,1,1
209,0,0,1,2,1,2,2,0,0,0,0,1,1,1,0,0,0,2,0,1


In [7]:
y_pred = metaest.predict(X_train)

In [8]:
from tpot.metrics import balanced_accuracy
b_acc = balanced_accuracy(y_true=y_train, y_pred=y_pred)
print(b_acc)

0.738899566846335


In [9]:
print(metaest.score(X_test, y_test))

0.5265151515151515


In [10]:
# metaestimator is passed when using it as a stand-alone estimator

In [11]:
# test metaestimator with cross_eval_score
cv_scores = cross_val_score(metaest, X, y, cv=3)
print(cv_scores)

[0.52247191 0.69101124 0.7481203 ]


In [12]:
# CV score passed

In [16]:
# Test metaestimator with TPOT with config

from tpot import TPOTClassifier


tpot_config = {
   'tpot.builtins.MetaEstimator': {
        
        'estimator': {
           'sklearn.ensemble.GradientBoostingRegressor': {
                'n_estimators': [100],
                'loss': ["ls", "lad", "huber", "quantile"],
                'learning_rate': [1e-3, 1e-2, 1e-1, 0.5, 1.],
                'max_depth': range(1, 11),
                'min_samples_split': range(2, 21),
                'min_samples_leaf': range(1, 21),
                'subsample': np.arange(0.05, 1.01, 0.05),
                'max_features': np.arange(0.05, 1.01, 0.05),
                'alpha': [0.75, 0.8, 0.85, 0.9, 0.95, 0.99]
            },

        },
       'A': [';'.join(A_feat_list)],
       'C': [';'.join(C_feat_list)]
    }
    
}


tpot = TPOTClassifier(generations=5, population_size=25, 
                        verbosity=2, cv=5,
                      config_dict=tpot_config,
                      template="Classifier",
                      scoring="balanced_accuracy",random_state=42)
tpot.fit(X_train, y_train)


A Jupyter Widget

Generation 1 - Current best internal CV score: 0.6860967622764402
Generation 2 - Current best internal CV score: 0.6860967622764402
Generation 3 - Current best internal CV score: 0.6860967622764402
Generation 4 - Current best internal CV score: 0.6946172049634054
Generation 5 - Current best internal CV score: 0.7144421114630568

Best pipeline: MetaEstimator(input_matrix, A=N10;N11;N12, C=N13;N14;N15, alpha=0.99, learning_rate=0.5, loss=ls, max_depth=2, max_features=0.8500000000000001, min_samples_leaf=16, min_samples_split=13, n_estimators=100, subsample=0.9000000000000001)
0.7211621357549285


In [17]:
print(tpot.score(X_test, y_test))

0.7211621357549285
